In [29]:
import os
from langchain_community.document_loaders import WebBaseLoader
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [16]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [2]:
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
print(loader)

In [5]:
document = loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuil

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_chunks = text_splitter.split_documents(document)
document_chunks

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain.

In [10]:
print(len(document_chunks))

24


In [17]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [20]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(document_chunks, embeddings)
vectorstore

In [28]:
query = "Many of the applications you build with LangChain"

result = vectorstore.similarity_search(query)
print(len(result))
for r in result:
    print(r.page_content)
    print('---')

4
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!
After reading this tutorial, you'll have a high level overview of:
---
storesWhy LangChain?Ecosystem🦜🛠️ LangSmith🦜🕸️ LangGraphVersionsv0.3v0.2Pydantic compatibilityMigrating from v0.0 chainsHow to migrate from v0.0 chainsMigrating from ConstitutionalChainMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDocumentsChainMigrating from RetrievalQAMigrating from StuffDocumentsCh

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on the provided context:
    <context>
    {context}
    </context>
    """
)

In [30]:
llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x169407140> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1694b9eb0> root_client=<openai.OpenAI object at 0x137d81dc0> root_async_client=<openai.AsyncOpenAI object at 0x1694b9220> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x169407140>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1694b9eb0>, root_client=<openai.OpenAI object at 0x137d81dc0>, root_async_client=<openai.AsyncOpenAI object at 0x1694b9220>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_nam

In [35]:
retriever = vectorstore.as_retriever()

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x134326900>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
    

In [37]:
result = retrieval_chain.invoke({'input':"Many of the applications you build with LangChain"})
result

{'input': 'Many of the applications you build with LangChain',
 'context': [Document(id='3a26f462-2eaf-488e-911d-5ab9cb6b410a', metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content="In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. St

In [39]:
print(result['answer'])

1. What is the primary purpose of the LLM application discussed in the context?

   The primary purpose of the LLM application discussed is to translate text from English into another language.

2. What is LangSmith used for in the context of building applications with LangChain?

   LangSmith is used for inspecting and logging traces of what is happening inside your chain or agent as your applications become more complex.

3. How can you install LangChain according to the provided context?

   You can install LangChain using pip with the command `pip install langchain` or using conda with the command `conda install langchain -c conda-forge`.

4. How do you enable tracing in a notebook environment for LangSmith?

   In a notebook environment, you can enable tracing by setting environment variables with the following code:
   ```python
   import getpass
   import os
   os.environ["LANGSMITH_TRACING"] = "true"
   os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
   ```
